## ANA - *sim* Spot Detection

### Notes

- In this notebook
    - Data prep (loading, cropping, time conversion)
    - Saving of prepped data for easy use in model fitting
    - Key plots of sim dynamics
    - Other bits and pieces (activation rates, comparison of measures)

### Prep

In [ ]:
# Imports

import os
import copy as pycopy
from collections import defaultdict

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib import lines

import skimage.io as io
import scipy.ndimage as ndi
import scipy.stats as stats

from ipywidgets import interact, widgets

import sys; sys.path.insert(0, '..')
import optonotch.utilities as utils

In [ ]:
# General settings

data_path = '../Data/Measurements/Sim'

experiments = ['continuous', 'late', 'pulsatile',
               'hairless-KO', 'hairless-Hpm', 'Twist-OE']

exp_colors  = {'continuous'   : 'teal',
               'late'         : 'darkorange',
               'pulsatile'    : 'orchid',
               'hairless-KO'  : 'firebrick',
               'hairless-Hpm' : 'firebrick',
               'Twist-OE'     : 'darkblue'}

In [ ]:
# Settings on cropping prior to gastrulation

crop_pre_rotation = True   # Cut-off per sample
crop_to_exp_maxtp = True   # Cut-off per experiment
crop_to_abs_maxtp = False  # Absolute overall cut-off
crop_to_shortest  = False  # Crop to shortest sample

exp_maxtps = {'continuous'   : 81,  # Manual max tp before gastrulation
              'late'         : 23,
              'pulsatile'    : 27,
              'hairless-KO'  : 65,
              'hairless-Hpm' : 65,
              'Twist-OE'     : 65}
abs_maxtp  = 80

# Note that the smallest value among those cropping steps set to True supercedes the others.

In [ ]:
# Helpful function for sample grouping

def get_exp(sample):
    
    for exp in experiments:
        
        if exp in sample:
            return exp

    raise Exception('No matching experiment found!')

### Data Preparation

#### Data Loading

In [ ]:
# Data loading function

def load_data(fpath):
    
    with open(fpath, 'r') as infile:
        header = infile.readline().strip().split('\t')
        full_arr = np.fromfile(infile, sep='\t').reshape((-1, len(header)))
    
    tp = []
    for t in range(int(np.max(full_arr[:,0]))):
        tp.append(full_arr[full_arr[:,0]==t])
        
    return header, tp

In [ ]:
# Find & load all relevant data

# Get all file paths
walk = list(os.walk(data_path))

# Create a dict for all samples
fpaths = []
for w in walk:
    if not '_new' in w[0]:
        for f in w[-1]:
            if f.endswith('_simSpots.tsv'):
                fpaths.append(os.path.join(w[0], f))

# Report
for fpath in fpaths:
    print(fpath)

# Load the data
sim_samples = []
sim_data = {}
for fpath in fpaths:
    fbase = os.path.split(fpath)[-1]
    fbase = fbase.replace('_simSpots.tsv', '')
    fbase = fbase.replace('early', 'continuous')  # Experiment was renamed for clarity
    sim_samples.append(fbase)
    header, sim_data[fbase] = load_data(fpath)
    
print('\n', header)

#### Data Cropping

Toward the end of each experiments, the embryos rotate/deform as grastrulation begins. The experiments need to be cut off before that.

In [ ]:
if crop_pre_rotation:  # Cut-off per sample
    
    # Manual cut-offs for each sample before gastrulation ruins measurements
    pre_rotation = {'1_continuous_sim timeseries stacks_zmax' : 81,
                    '2_continuous_sim timeseries stacks_zmax' : 66,
                    '3_continuous_sim timeseries stacks_zmax' : 67,
                    '4_continuous_sim timeseries stacks_zmax' : 119,
                    '5_continuous_sim timeseries stacks_zmax' : 100,
                    '6_continuous_sim timeseries stacks_zmax' : 75,
                    '7_continuous_sim timeseries stacks_zmax' : 80,
                    '1_late_sim timeseries stacks_zmax' : 25,
                    '2_late_sim timeseries stacks_zmax' : 26,
                    '3_late_sim timeseries stacks_zmax' : 26,
                    '4_late_sim timeseries stacks_zmax' : 21,
                    '5_late_sim timeseries stacks_zmax' : -1,
                    '1_pulsatile_sim timeseries stacks_zmax' : -1,
                    '2_pulsatile_sim timeseries stacks_zmax' : -1,
                    '3_pulsatile_sim timeseries stacks_zmax' : -1,
                    '4_pulsatile_sim timeseries stacks_zmax' : -1,
                    '5_pulsatile_sim timeseries stacks_zmax' : -1,
                    '1_hairless-KO_H2_sim timeseries stacks_zmax' : 65,
                    '2_hairless-KO_H2_sim timeseries stacks_zmax' : -1,
                    '3_hairless-KO_H2_sim timeseries stacks_zmax' : 68,
                    '4_hairless-KO_H2_sim timeseries stacks_zmax' : 66,
                    '5_hairless-KO_H2_sim timeseries stacks_zmax' : 66,
                    '1_hairless-Hpm_sim timeseries stacks_zmax' : 70,
                    '2_hairless-Hpm_sim timeseries stacks_zmax' : 80,
                    '3_hairless-Hpm_sim timeseries stacks_zmax' : 74,
                    '4_hairless-Hpm_sim timeseries stacks_zmax' : 70,
                    '5_hairless-Hpm_sim timeseries stacks_zmax' : 70,
                    '6_hairless-Hpm_sim timeseries stacks_zmax' : 70,
                    '1_Twist-OE_sim timeseries stacks_zmax' : 74,
                    '2_Twist-OE_sim timeseries stacks_zmax' : 67,
                    '3_Twist-OE_sim timeseries stacks_zmax' : 65,
                    '4_Twist-OE_sim timeseries stacks_zmax' : 65,
                    '5_Twist-OE_sim timeseries stacks_zmax' : 70}
    
    for sim_sample in sim_data:
        sim_data[sim_sample] = sim_data[sim_sample][:pre_rotation[sim_sample]]

In [ ]:
if crop_to_exp_maxtp:  # Cut-off per experiment
    
    for sim_sample in sim_data:
        label = get_exp(sim_sample)
        sim_data[sim_sample] = sim_data[sim_sample][:exp_maxtps[label]]

In [ ]:
if crop_to_abs_maxtp:  # Absolute overall cut-of
    
    for sim_sample in sim_data:
        if len(sim_data[sim_sample]) > abs_maxtp:
            sim_data[sim_sample] = sim_data[sim_sample][:abs_maxtp]

In [ ]:
if crop_to_shortest:  # Crop to shortest sample
    
    n_tps = defaultdict(lambda : [])
    for sim_sample in sim_data:
        label = get_exp(sim_sample)
        n_tps[label].append(len(sim_data[sim_sample]))
    
    for sim_sample in sim_data:
        sim_data[sim_sample] = sim_data[sim_sample][:min(n_tps[get_exp(sim_sample)])]

#### Other prep & checks

In [ ]:
# Prep time vectors (info from Ranjith)

time = {}
for sim_sample in sim_samples:
    
    # Most experiments have half-minute resolutions
    if any(exp in sim_sample for exp in ['continuous', 'late',
                                         'hairless-KO', 'hairless-Hpm',
                                         'Twist-OE']):
        t = np.arange(len(sim_data[sim_sample])) * 0.5
    
    # The pulsatile is obviously a special case
    elif 'pulsatile' in sim_sample:
        t = np.zeros(len(sim_data[sim_sample]))
        c = 0
        for tp in np.arange(len(sim_data[sim_sample])):
            if tp==0:
                t[tp] = 0.0
            elif c==10:
                t[tp] = t[tp-1] + 10.0
                c = 0
            else:
                t[tp] = t[tp-1] + 0.5
            c += 1

    # Default for other cases is minute resolution
    else:
        t = np.arange(len(sim_data[sim_sample]))
    
    time[sim_sample] = t

### Save Prepared Data for Model Fitting

In [ ]:
# Group samples in experiments
sample_exp_dict = defaultdict(lambda : [])
for sim_sample in sim_data:
    sample_exp_dict[get_exp(sim_sample)].append(sim_sample)

# Create output file
with open('../Data/Measurements/Sim/compiled_count_data.csv', 'w') as outfile:
    
    for experiment in sample_exp_dict:
        for sim_sample in sample_exp_dict[experiment]:
            
            # Prepare data
            sample_times  = time[sim_sample]
            sample_counts = np.array([sim_data[sim_sample][t].shape[0] 
                                     for t in range(len(sim_data[sim_sample]))])
            sample_totalints = np.array([sim_data[sim_sample][t][:,5].sum()
                                         if sim_data[sim_sample][t].shape[1]>0 else 0.0
                                         for t in range(len(sim_data[sim_sample]))])
            
            # Write to file
            outfile.write(experiment + '\t' + sim_sample + '\t' + 'TIMES:' + '\t' +
                          '\t'.join(sample_times.astype(str)) + '\n')
            
            outfile.write(experiment + '\t' + sim_sample + '\t' + 'COUNTS:' + '\t' +
                          '\t'.join(sample_counts.astype(str)) + '\n')
            
            outfile.write(experiment + '\t' + sim_sample + '\t' + 'TOTALINTS:' + '\t' +
                          '\t'.join(sample_totalints.astype(str)) + '\n')
            
            # Report
            print(experiment, sim_sample, len(sample_times), len(sample_counts), len(sample_totalints))

### Key Plots of Sim Dynamics

In [ ]:
# Dynamics under continuous, late or pulsatile activation

# Initialize
fig, axs = plt.subplots(3, 1, figsize=(9.5*1.0, 10*1.0))

# Get axes
ax_dict = {'continuous' : axs[0],
           'late' : axs[1],
           'pulsatile' : axs[2]}

# Select relevant sample names
selected = [spl for spl in sim_samples
            if spl.split('_')[1] in ['continuous', 'late', 'pulsatile']
            and not '2_continuous' in spl] #*
            # *preserve accidental exclusion due to file naming error

# For summary stats
all_x  = {}
all_y  = {}

# For the legend
labels = []

# Plot individual samples
for sim_sample in selected:

    # Get the data
    plot_x = time[sim_sample]
    plot_y = np.array([sim_data[sim_sample][t].shape[0]  # Compute spot count
                       for t in range(len(sim_data[sim_sample]))])

    # Shift late experiment
    if 'late' in sim_sample:
        plot_x = plot_x + 25

    # Get stuff
    label = get_exp(sim_sample)
    color = exp_colors[label]
    ax = ax_dict[label]

    # Some axis cosmetics...
    ax.tick_params(axis='both', which='major', labelsize=14)
    [tick.set_fontname('Arial') for tick in ax.get_xticklabels()]
    [tick.set_fontname('Arial') for tick in ax.get_yticklabels()]

    # Plot straightforward cases
    if label != 'pulsatile':
        ax.plot(plot_x, plot_y, color=color, 
                alpha=0.5, 
                label='_no-label_',
                path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])

    # Plot pulsatile case
    else:
        ax.plot(plot_x[:10], plot_y[:10], color=color, 
                alpha=0.5, label='_no-label_',
                path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])
        ax.plot(plot_x[9:11], plot_y[9:11], color=color, alpha=0.3, linestyle='--',
                path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])
        ax.plot(plot_x[10:20], plot_y[10:20], color=color, alpha=0.7,
                path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])
        ax.plot(plot_x[19:21], plot_y[19:21], color=color, alpha=0.3, linestyle='--',
                path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])
        ax.plot(plot_x[20:30], plot_y[20:30], color=color, alpha=0.7,
                path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])

    # Handle legend labels
    make_legend = False
    if label not in labels:
        make_legend = True
        labels.append(label)
        all_x[label] = []
        all_y[label] = []

    # Compile data for summary stats
    all_x[label].append(plot_x)
    all_y[label].append(plot_y)

    # Add y axis label
    ax.set_ylabel('number of SIM spots', fontsize=15, fontname='Arial')

# Add running mean
for label in labels:

    ax = ax_dict[label]
    mean_x, mean_y = utils.running_mean(np.concatenate(all_x[label]), 
                                        np.concatenate(all_y[label]), 
                                        window=0.5)

    if label != 'pulsatile':
        ax.plot(mean_x, mean_y, label='mean',
                color=exp_colors[label], lw=2.0,
                path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])

    else:
        ax.plot(mean_x[:10], mean_y[:10], label='mean',
                color=exp_colors[label], lw=2.0,
                path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])
        ax.plot(mean_x[ 9:11], mean_y[9:11], color=exp_colors[label], lw=2.0, 
                alpha=0.7, linestyle='--',
                path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])
        ax.plot(mean_x[10:20], mean_y[10:20], color=exp_colors[label], lw=2.0,
                path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])
        ax.plot(mean_x[19:21], mean_y[19:21], color=exp_colors[label], lw=2.0, 
                alpha=0.7, linestyle='--',
                path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])
        ax.plot(mean_x[20:30], mean_y[20:30], color=exp_colors[label], lw=2.0,
                path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])

# Add running CI
for label in labels:
    ax = ax_dict[label]
    ci_x, ci_y = utils.running_CI(np.concatenate(all_x[label]), 
                                  np.concatenate(all_y[label]), 
                                  ci_type='single', window=0.0, min_samples=5)
    ax.fill_between(ci_x, ci_y[:,0], ci_y[:,1], 
                    color=exp_colors[label], alpha=0.1, linewidth=0.0,
                    label='CI95')    

# Compute x axis limits
all_x_flat = np.concatenate([np.concatenate(all_x[l]) for l in labels])
all_x_flat = all_x_flat[~np.isnan(all_x_flat)]
for ax in ax_dict.values():
    ax.set_xlim([np.min(all_x_flat)-1, np.max(all_x_flat)+1])

# Compute y axis limits
all_y_flat = np.concatenate([np.concatenate(all_y[l]) for l in labels])
all_y_flat = all_y_flat[~np.isnan(all_y_flat)]
for ax in ax_dict.values():
    ax.set_ylim([np.min(all_y_flat)-0, np.max(all_y_flat)+30])

# Add x label
axs[2].set_xlabel('time [min]', fontsize=15, fontname='Arial')

# Add experiment label
for label in labels:
    ax = ax_dict[label]
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.text(xlim[0] + 0.5, ylim[1] - 18, 
            label, ha='left', va='top', 
            fontsize=14, weight='bold',
            fontname='Arial',
            color=exp_colors[label])

# Add legend
for label in labels:
    ax = ax_dict[label]
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.legend(loc='upper left', bbox_to_anchor=(0.0,0.85), 
              prop={'family':'Arial', 'size':13})

# Add activation indicator
for label in labels:
    ax = ax_dict[label]
    plot_x = all_x[label][np.argmax([len(p_x) for p_x in all_x[label]])]
    ylim = ax.get_ylim()
    xlim = ax.get_xlim()
    if label!='pulsatile':
        activation_windows = [[plot_x[0], min([xlim[1], plot_x[-1]])]]
    else:
        activation_windows = [[plot_x[0],  plot_x[9] ],
                              [plot_x[10], plot_x[19]],
                              [plot_x[20], min(xlim[1], plot_x[26])]]
    for aw in activation_windows:
        line_x = [aw[0], aw[-1]]
        line_y = [ylim[1]-(ylim[1]-ylim[0])/40, ylim[1]-(ylim[1]-ylim[0])/40]
        line = lines.Line2D(line_x, line_y, lw=5., color='#0F76BB')
        line.set_clip_on(False)
        ax.add_line(line)

# Finalize
plt.tight_layout()
plt.savefig(r'../Figures/2_ABC.pdf')
plt.show()

In [ ]:
# Dynamics under continuous, late or pulsatile activation

# Initialize
fig, axs = plt.subplots(3, 1, figsize=(9.5*1.0, 10*1.0))

# Get axes
ax_dict = {'hairless-KO' : axs[0],
           'hairless-Hpm' : axs[1],
           'Twist-OE' : axs[2]}

# Select relevant sample names
selected = [spl for spl in sim_samples
            if spl.split('_')[1] in ['hairless-KO', 'hairless-Hpm', 'Twist-OE']]

# For summary stats
all_x  = {}
all_y  = {}

# For the legend
labels = []

# Plot individual samples
for sim_sample in selected:

    # Get the data
    plot_x = time[sim_sample]
    plot_y = np.array([sim_data[sim_sample][t].shape[0]  # Compute spot count
                       for t in range(len(sim_data[sim_sample]))])

    # Get stuff
    label = get_exp(sim_sample)
    color = exp_colors[label]
    ax = ax_dict[label]

    # Some axis cosmetics...
    ax.tick_params(axis='both', which='major', labelsize=14)
    [tick.set_fontname('Arial') for tick in ax.get_xticklabels()]
    [tick.set_fontname('Arial') for tick in ax.get_yticklabels()]

    # Plot
    ax.plot(plot_x, plot_y, color=color, 
            alpha=0.5, 
            label='_no-label_',
            path_effects=[pe.Stroke(linewidth=4.0, foreground='w'), pe.Normal()])

    # Handle legend labels
    make_legend = False
    if label not in labels:
        make_legend = True
        labels.append(label)
        all_x[label] = []
        all_y[label] = []

    # Compile data for summary stats
    all_x[label].append(plot_x)
    all_y[label].append(plot_y)

    # Add y axis label
    ax.set_ylabel('number of SIM spots', fontsize=15, fontname='Arial')

# Add running mean
for label in labels:

    ax = ax_dict[label]
    mean_x, mean_y = utils.running_mean(np.concatenate(all_x[label]), 
                                        np.concatenate(all_y[label]), 
                                        window=0.5)

    ax.plot(mean_x, mean_y, label='mean',
            color=exp_colors[label], lw=2.0,
            path_effects=[pe.Stroke(linewidth=4.5, foreground='w'), pe.Normal()])

# Add running CI
for label in labels:
    ax = ax_dict[label]
    ci_x, ci_y = utils.running_CI(np.concatenate(all_x[label]), 
                                  np.concatenate(all_y[label]), 
                                  ci_type='single', window=0.0, min_samples=5)
    ax.fill_between(ci_x, ci_y[:,0], ci_y[:,1], 
                    color=exp_colors[label], alpha=0.1, linewidth=0.0,
                    label='CI95')    

# Compute x axis limits
all_x_flat = np.concatenate([np.concatenate(all_x[l]) for l in labels])
all_x_flat = all_x_flat[~np.isnan(all_x_flat)]
for ax in ax_dict.values():
    ax.set_xlim([np.min(all_x_flat)-1, np.max(all_x_flat)+1])

# Compute y axis limits
all_y_flat = np.concatenate([np.concatenate(all_y[l]) for l in labels])
all_y_flat = all_y_flat[~np.isnan(all_y_flat)]
for akey, ax in ax_dict.items():
    if akey=='hairless-Hpm':  # Separate for EV5
        ax.set_ylim([np.min(all_y_flat)-0, np.max(all_y_flat)+70])
    else:
        ax.set_ylim([np.min(all_y_flat)-0, np.max(all_y_flat)+30])

# Add x label
axs[2].set_xlabel('time [min]', fontsize=15, fontname='Arial')

# Add experiment label
for label in labels:
    ax = ax_dict[label]
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    if label=='hairless-Hpm':  # Separate for EV5
        ax.text(xlim[1] - 1.2, ylim[1] - 18, 
                label, ha='right', va='top', 
                fontsize=14, weight='bold',
                fontname='Arial',
                color=exp_colors[label])
    else:
        ax.text(xlim[0] + 0.5, ylim[1] - 18, 
                label, ha='left', va='top', 
                fontsize=14, weight='bold',
                fontname='Arial',
                color=exp_colors[label])

# Add legend
for label in labels:
    ax = ax_dict[label]
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    if label=='hairless-Hpm':  # Separate for EV5
        ax.legend(loc='upper right', bbox_to_anchor=(0.98,0.90), 
                  prop={'family':'Arial', 'size':13})
    else:
        ax.legend(loc='upper left', bbox_to_anchor=(0.0,0.90), 
                  prop={'family':'Arial', 'size':13})

# Add activation indicator
for label in labels:
    ax = ax_dict[label]
    plot_x = all_x[label][np.argmax([len(p_x) for p_x in all_x[label]])]
    ylim = ax.get_ylim()
    xlim = ax.get_xlim()
    activation_windows = [[plot_x[0], min([xlim[1], plot_x[-1]])]]
    for aw in activation_windows:
        line_x = [aw[0], aw[-1]]
        line_y = [ylim[1]-(ylim[1]-ylim[0])/-40, ylim[1]-(ylim[1]-ylim[0])/-40]
        line = lines.Line2D(line_x, line_y, lw=5., color='#0F76BB')
        line.set_clip_on(False)
        ax.add_line(line)

# Finalize
plt.tight_layout()
plt.savefig(r'../Figures/4_DE__EV5.pdf')
plt.show()

### Other Bits and Pieces

#### Simple quantification of activation and adaptation rates

In [ ]:
# Results for each experiment
exp_activ_slopes = {}
exp_adapt_slopes = {}

for exp in experiments:
    
    # Skip pulsatile
    if exp=='pulsatile':
        continue
    
    activation_slopes = []
    adaptation_slopes = []
    
    for sim_sample in sim_samples:
        if get_exp(sim_sample) == exp:
            
            # Compute spot count
            c = np.array([sim_data[sim_sample][t].shape[0]
                          for t in range(len(sim_data[sim_sample]))])
            t = time[sim_sample]
            
            # Compute activation rate
            ac_slope = np.gradient(c[t<=15], t[t<=15]).max()
            
            # Compute adaptation rate
            if exp=='late':
                ad_slope = np.nan
            else:
                ad_slope = np.gradient(c[t>min(max(t),15)], t[t>min(max(t),15)]).mean()
            
            activation_slopes.append(ac_slope)
            adaptation_slopes.append(ad_slope)
            
    exp_activ_slopes[exp] = activation_slopes
    exp_adapt_slopes[exp] = adaptation_slopes

# Report results
print('Activation Slope\n----------------')
for exp in exp_activ_slopes: 
    print('{:<12}'.format(exp), end='  ')
    print('mu={:.3f}'.format(np.mean(exp_activ_slopes[exp])), end='  ')
    print('sd={:.3f}'.format(np.std(exp_activ_slopes[exp])))

print('\nAdaptation Slope\n----------------')
for exp in exp_adapt_slopes:
    print('{:<12}'.format(exp), end='  ')
    print('mu={:.3f}'.format(np.mean(exp_adapt_slopes[exp])), end='  ')
    print('sd={:.3f}'.format(np.std(exp_adapt_slopes[exp])))

#### Check consistency of the two *sim* measures

In [ ]:
plt.figure(figsize=(6,6))

plot_labels = []
for sim_sample in sim_data:
        
    # Spot counts
    plot_x = np.array([sim_data[sim_sample][t].shape[0] 
                       for t in range(len(sim_data[sim_sample]))])

    # Total spot intensity
    plot_y = np.array([sim_data[sim_sample][t][:,5].sum()
                       for t in range(len(sim_data[sim_sample]))])

    # Plot
    plot_label = get_exp(sim_sample)
    if plot_label=='hairless-KO':
        plot_label = 'hairless-H2'
    if plot_label=='hairless-Hpm':
        plot_label = 'hairless-H(LD)'
    plt.scatter(plot_x, plot_y, c=exp_colors[get_exp(sim_sample)], 
                alpha=0.25, s=5, 
                label=plot_label if plot_label not in plot_labels else '__no_label__')
    plot_labels.append(plot_label)

# Legend
handles, labels = plt.gca().get_legend_handles_labels()
order = [0,3,4,1,2,5]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
           fontsize=13)

# Labels
plt.title('Comparison of measures\nof $sim$ expression', fontsize=16)
plt.xlabel('number of $sim$ spots', fontsize=14, labelpad=7)
plt.ylabel('$sim$ expression\n(total spot intensity)', fontsize=14, labelpad=10)
plt.gca().tick_params(axis='both', which='major', labelsize=14)

# Finalize
plt.tight_layout()
plt.savefig(r'../Figures/EV6.pdf')
plt.show()